# Class and Magic Methods
---
## Class
### private and protected

In [1]:
class Base:
    def __init__(self, public, protected, private):
        self.public = public
        self._protected = protected
        self.__private = private
        print("Access attributes inside the class")
        print(f"public variable: {self.public}")
        print(f"protected variable: {self._protected}")
        print(f"private variable: {self.__private}")

base = Base('public variable', 'protected variable', 'private variable')
print(base.public)
print(base._protected)

# access a private variable from outside its class will result in an AttributeError
# print(base.__private) 

print(base._Base__private)

Access attributes inside the class
public variable: public variable
protected variable: protected variable
private variable: private variable
public variable
protected variable
private variable


### property() function

```
Signatures:
prop = property(getter, setter, deleter, docstring)
```

In [2]:
class Person:
    def __init__(self, name='Anonymous'):
        self.__name = name

    def setname(self, name):
        self.__name = name
        
    def getname(self):
        return self.__name
    
p1 = Person()
print(f"default name: {p1.getname()}")
p1.setname('John')
print(f"new name: {p1.getname()}")

class PersonWithProperty:
    """
    property(getname, setname) returns the property object and assigns it to name.
    The name property hides the private instace attribute __name. The name property is
    accessed directly, but internally it will invoke the getname() or setname() method.
    """
    
    def __init__(self):
        self.__name = ''
        
    def setname(self, name):
        print('setname() called')
        self.__name = name
    
    def getname(self):
        print('getname() called')
        return self.__name
    
    def delname(self):
        print('delname() called')
        del self.__name
    
    name = property(getname, setname, delname)

p2 = PersonWithProperty()
p2.name = "Jane"
print(f"new name: {p2.name}")
del p2.name

default name: Anonymous
new name: John
setname() called
getname() called
new name: Jane
delname() called


### @property decorator

A decorator is a function that receives another function as argument. The behavior of the argument function is extended by the decorator without actually modifying it.

@property decorator is a built-in decorator in Python for the property() function.

### @classmethod decorator and @staticmethod decorator

- The @classmethod decorator can be applied on any method of a class. 
- The @staticmethod is a built-in decorator that defines a static method in the class. A static method doesn't receive any reference argument whether it is called by an instance of a class or by the class itself. -> doesn't have any arguments - neither `self` nor `cls` 

In [3]:
def mydecoratorfunction(some_function): # function to be decorated passed as argument
    def wrapper_function(): # wrap the some_function and extends its behaviour
        # write code to extend the behaviour of some_function()
        some_function() # call some_function
        return wrapper_function # return wrapper function

def display(string):
    print(string)
    
def displaydecorator(fn):
    def display_wrapper(string):
        print('Output:', end=" ")
        fn(string)
    return display_wrapper

out = displaydecorator(display)
out('Hello World')

@displaydecorator
def display2(string):
    print(f'{string} via @displaydecorator')
    
display2('Hello World')


class PersonWithPropertyDecorator:
    total_objects = 0

    def __new__(cls):
        cls.total_objects += 1
        instance = object.__new__(cls)
        return instance

    def __init__(self):
        self.__name = ''
        # an alternative for not using __new__ magic method
        # PersonWithPropertyDecorator.total_objects += 1
        
    @property
    def name(self):
        return self.__name
    
    @name.setter
    def name(self, name):
        self.__name = name
        
    @name.deleter
    def name(self):
        print('Deleting..')
        del self.__name
        PersonWithPropertyDecorator.total_objects -= 1
        
    @classmethod
    def show_count(cls):
        print(f"Total persons: {cls.total_objects}")

    @staticmethod
    def greet():
        print('Hola!')
        
p3 = PersonWithPropertyDecorator()
p3.name = 'Steve'
print(f"get name {p3.name}")
PersonWithPropertyDecorator.show_count() # classmethod can be called using the class name
del p3.name
# p3.name # An AttributeError
p3.show_count() # classmethod can also be called using the object
p4 = PersonWithPropertyDecorator()
p4.show_count()
p5 = PersonWithPropertyDecorator()
p5.show_count()
PersonWithPropertyDecorator.greet() # staticmethod can be called using both the class name and the object
p5.greet()

Output: Hello World
Output: Hello World via @displaydecorator
get name Steve
Total persons: 1
Deleting..
Total persons: 0
Total persons: 1
Total persons: 2
Hola!
Hola!


# Python Magic Methods

In [4]:
class Base:
    pass

dir(Base)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__']

## __new__() method

In Python the `__new__()` magic method is implicitly called before the `__init__()` method. The `__new__()` method returns a new object, which is then initialized by `__init__()`.

In [5]:
class Base:
    def __new__(cls):
        print("__new__ magic method is called")
        instance = object.__new__(cls)
        return instance
    
    def __init__(self):
        print("__init__ magic method is called")

Base()

__new__ magic method is called
__init__ magic method is called


## References
1. [Python - Class from TutorialsTeacher](https://www.tutorialsteacher.com/python/python-class)
1. [Python - Magic Methods from TutorialsTeacher](https://www.tutorialsteacher.com/python/magic-methods-in-python)
2. [wfitz / Python / Classes.ipynb](https://github.com/wfitz/Python/blob/master/Classes.ipynb)